![header](../figures/logos_partenaires._cmems_se.jpg)


<div class="alert alert-block alert-success">
<h1><center>Demo: Create and save mp4 movies of SWOT data  </center></h1></div>

***
**Authors:**  Datlas, IGE, IMEDEA & CLS <br>
**Copyright:** 2022 Datlas, IGE, IMEDEA & CLS <br>
**License:** MIT

In [1]:
import os
import glob
import sys
sys.path.append('..')
import numpy as np 
import xarray as xr 
import matplotlib.pyplot as plt
from datetime import datetime
import glob 

from src import mod_plot
from src import mod_switchvar
from src.dict_regions_info import *

In [2]:

region_info = RegionInfo('GS_info').region_info   # Region dictionary info from dict_regions_info.py


<div class="alert alert-info" role="alert">

<h2>1. Read from regional data file </h2>

</div>

From: "../swot_data/swot_1j_share_'region'.nc"

In [3]:
path_data = '../swot_data/swot_1j_share' 
  
all_files = sorted(glob.glob(path_data+'_'+region_info['name']+'.nc'))

ds_passes = xr.open_mfdataset(all_files,combine='nested',concat_dim='num_lines')
ds_passes

<xarray.Dataset>
Dimensions:             (num_lines: 12222, num_pixels: 69)
Coordinates:
    latitude            (num_lines, num_pixels) float32 dask.array<chunksize=(12222, 69), meta=np.ndarray>
    longitude           (num_lines, num_pixels) float32 dask.array<chunksize=(12222, 69), meta=np.ndarray>
Dimensions without coordinates: num_lines, num_pixels
Data variables:
    time                (num_lines, num_pixels) datetime64[ns] dask.array<chunksize=(12222, 69), meta=np.ndarray>
    mss                 (num_lines, num_pixels) float32 dask.array<chunksize=(12222, 69), meta=np.ndarray>
    swh_model           (num_lines, num_pixels) float32 dask.array<chunksize=(12222, 69), meta=np.ndarray>
    ssha_new_editing    (num_lines, num_pixels) float32 dask.array<chunksize=(12222, 69), meta=np.ndarray>
    new_quality_flag    (num_lines, num_pixels) float64 dask.array<chunksize=(12222, 69), meta=np.ndarray>
    ssha_denoised_unet  (num_lines, num_pixels) float32 dask.array<chunksize=(12222, 69), meta=np.ndarray>
Attributes: (12/41)
    contact:                         aviso@altimetry.fr
    creator_email:                   aviso@altimetry.fr
    creator_name:                    DUACS - Data Unification and Altimeter C...
    creator_url:                     https://aviso.altimetry.fr
    institution:                     CNES
    license:                         https://www.aviso.altimetry.fr/fileadmin...
    ...                              ...
    time_coverage_begin:             2023-04-23T17:58:31Z
    time_coverage_end:               2023-04-23T18:49:36Z
    geospatial_lat_min:              -78.272196
    geospatial_lat_max:              78.272247
    geospatial_lon_min:              8.4e-05
    geospatial_lon_max:              359.999504

<div class="alert alert-info" role="alert">

<h2>2. Create SSH movie </h2>

</div> 

In [4]:
mod_plot.create_movie(ds_passes, var_type = 'ssh',name_denoised='ssha_denoised_unet', method='Unet_baseline', region_info= region_info, time0 = np.datetime64('2023-04-23'),
                 ndays = 12, dir_output='../figures/', dim_name=['time','latitude','longitude'], 
                 framerate=3, Display=True, newmovie=True)

2023-04-23
2023-04-24
2023-04-25
2023-04-26
2023-04-27
2023-04-28
2023-04-29
2023-04-30
2023-05-01
2023-05-02
2023-05-03
2023-05-04


<div class="alert alert-info" role="alert">

<h2>3. Compute SSH gradients and Laplacian </h2>

</div>
 

In [5]:
grad_new_editing, lapl_new_editing = mod_switchvar.deriv1and2(ds_passes.ssha_new_editing)
grad_denoised_unet, lapl_denoised_unet = mod_switchvar.deriv1and2(ds_passes.ssha_denoised_unet)

ds_passes = ds_passes.assign(grad_new_editing=lambda ds_passes: ds_passes.ssha_new_editing * 0 + grad_new_editing)
ds_passes = ds_passes.assign(grad_denoised_unet=lambda ds_passes: ds_passes.ssha_denoised_unet * 0 + grad_denoised_unet)

ds_passes = ds_passes.assign(lapl_new_editing=lambda ds_passes: ds_passes.ssha_new_editing * 0 + lapl_new_editing)
ds_passes = ds_passes.assign(lapl_denoised_unet=lambda ds_passes: ds_passes.ssha_denoised_unet * 0 + lapl_denoised_unet)


<div class="alert alert-info" role="alert">

<h2>4. Create SSH gradients and Laplacian movies </h2>

</div>
 

In [6]:
mod_plot.create_movie(ds_passes, var_type = 'grad',name_denoised='grad_denoised_unet', method='Unet_baseline', region_info= region_info, time0 = np.datetime64('2023-04-23'),
                 ndays = 12, dir_output='../figures/', dim_name=['time','latitude','longitude'], 
                 framerate=3, Display=True, newmovie=True)

2023-04-23
2023-04-24
2023-04-25
2023-04-26
2023-04-27
2023-04-28
2023-04-29
2023-04-30
2023-05-01
2023-05-02
2023-05-03
2023-05-04


In [7]:
mod_plot.create_movie(ds_passes, var_type = 'lapl',name_denoised='lapl_denoised_unet', method='Unet_baseline', region_info= region_info, time0 = np.datetime64('2023-04-23'),
                 ndays = 12, dir_output='../figures/', dim_name=['time','latitude','longitude'], 
                 framerate=3, Display=True, newmovie=True)

2023-04-23
2023-04-24
2023-04-25
2023-04-26
2023-04-27
2023-04-28
2023-04-29
2023-04-30
2023-05-01
2023-05-02
2023-05-03
2023-05-04
